In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings 
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from catboost import CatBoostClassifier




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    obesity_order = {
        'Insufficient_Weight': 0,
        'Normal_Weight': 1,
        'Overweight_Level_I': 2,
        'Overweight_Level_II': 3,
        'Obesity_Type_I': 4,
        'Obesity_Type_II': 5,
        'Obesity_Type_III': 6
    }
    # Consumption of alcohol (CALC
    calc_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2    
    }

    #  Calories consumption monitoring (SCC)
    yes_no_order={'no':0,'yes':1}

    # Consumption of food between meals (CAEC)
    caec_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    }

    transpot_order={
        'Walking':0,
        'Bike':1,
        'Motorbike':2,
        'Public_Transportation':3,
        'Automobile':4

    }
    
    gender_order={'Female':0,'Male':1}
    
    df['Gender']=df['Gender'].replace(gender_order).astype('category')
    df['NObeyesdad']=df['NObeyesdad'].replace(obesity_order).astype('category')
    df['CALC']=df['CALC'].replace(calc_order).astype('category')
    df['SCC']=df['SCC'].replace(yes_no_order).astype('category')
    df['SMOKE']=df['SMOKE'].replace(yes_no_order).astype('category')
    df['family_history_with_overweight']=df['family_history_with_overweight'].replace(yes_no_order).astype('category')
    df['FAVC']=df['FAVC'].replace(yes_no_order).astype('category')
    df['CAEC']=df['CAEC'].replace(caec_order).astype('category')
    df['MTRANS']=df['MTRANS'].replace(transpot_order).astype('category')
    
    return df

In [3]:
df=wrangle("/kaggle/input/playground-series-s4e2/train.csv")
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1,0,2.763573,0,0.000000,0.976473,1,3,3
1,1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2,0,2.000000,0,1.000000,1.000000,0,4,1
2,2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1,0,1.910378,0,0.866045,1.673584,0,3,0
3,3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1,0,1.674061,0,1.467863,0.780199,1,3,6
4,4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1,0,1.979848,0,1.967973,0.931721,1,3,3


In [4]:
target_column = 'NObeyesdad'
X = df.drop(['id','NObeyesdad'], axis=1)
y = df[target_column]

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ['Gender','CALC','SCC','SMOKE','family_history_with_overweight','FAVC','CAEC','MTRANS']
# Initialize the CatBoost Classifier
catboost_model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='MultiClass',cat_features=categorical_features)

# Train the model
catboost_model.fit(X_train, y_train)



0:	learn: 1.6248053	total: 196ms	remaining: 3m 15s
1:	learn: 1.4195498	total: 338ms	remaining: 2m 48s
2:	learn: 1.2760225	total: 475ms	remaining: 2m 37s
3:	learn: 1.1611720	total: 615ms	remaining: 2m 33s
4:	learn: 1.0699959	total: 761ms	remaining: 2m 31s
5:	learn: 0.9951728	total: 904ms	remaining: 2m 29s
6:	learn: 0.9356175	total: 1.05s	remaining: 2m 28s
7:	learn: 0.8815642	total: 1.19s	remaining: 2m 27s
8:	learn: 0.8338495	total: 1.33s	remaining: 2m 26s
9:	learn: 0.7880849	total: 1.5s	remaining: 2m 28s
10:	learn: 0.7475527	total: 1.68s	remaining: 2m 31s
11:	learn: 0.7152909	total: 1.85s	remaining: 2m 32s
12:	learn: 0.6836417	total: 1.99s	remaining: 2m 31s
13:	learn: 0.6553122	total: 2.13s	remaining: 2m 30s
14:	learn: 0.6291450	total: 2.28s	remaining: 2m 29s
15:	learn: 0.6097139	total: 2.42s	remaining: 2m 28s
16:	learn: 0.5901413	total: 2.56s	remaining: 2m 27s
17:	learn: 0.5738362	total: 2.7s	remaining: 2m 27s
18:	learn: 0.5572329	total: 2.85s	remaining: 2m 27s
19:	learn: 0.5400609	tot

In [6]:
# Make predictions on the test set
y_pred = catboost_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.904383429672447


# Prediction

In [7]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    obesity_order = {
        'Insufficient_Weight': 0,
        'Normal_Weight': 1,
        'Overweight_Level_I': 2,
        'Overweight_Level_II': 3,
        'Obesity_Type_I': 4,
        'Obesity_Type_II': 5,
        'Obesity_Type_III': 6
    }
    # Consumption of alcohol (CALC
    calc_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    }

    #  Calories consumption monitoring (SCC)
    yes_no_order={'no':0,'yes':1}

    # Consumption of food between meals (CAEC)
    caec_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    }

    transpot_order={
        'Walking':0,
        'Bike':1,
        'Motorbike':2,
        'Public_Transportation':3,
        'Automobile':4

    }
    
    gender_order={'Female':0,'Male':1}
    
    #drop id
    df=df.drop(['id'], axis=1)
    
    df['Gender']=df['Gender'].replace(gender_order).astype('category')
    df['CALC']=df['CALC'].replace(calc_order).astype('category')
    df['SCC']=df['SCC'].replace(yes_no_order).astype('category')
    df['SMOKE']=df['SMOKE'].replace(yes_no_order).astype('category')
    df['family_history_with_overweight']=df['family_history_with_overweight'].replace(yes_no_order).astype('category')
    df['FAVC']=df['FAVC'].replace(yes_no_order).astype('category')
    df['CAEC']=df['CAEC'].replace(caec_order).astype('category')
    df['MTRANS']=df['MTRANS'].replace(transpot_order).astype('category')
    
    #predicton
    y_pred = catboost_model.predict(df)
    
    
    
    return y_pred

In [8]:
prediction=wrangle("/kaggle/input/playground-series-s4e2/test.csv")

In [9]:
test_df=pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")


In [10]:
submission_df=pd.DataFrame({
    'id': test_df['id'],  # Assuming 'id' is a column in your original DataFrame
    'NObeyesdad': prediction.flatten()
})

In [11]:
obesity_order = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Overweight_Level_I': 2,
    'Overweight_Level_II': 3,
    'Obesity_Type_I': 4,
    'Obesity_Type_II': 5,
    'Obesity_Type_III': 6
}

In [12]:
# convert numercial value into orginal ordinal value
submission_df['NObeyesdad'] = submission_df['NObeyesdad'].map({v: k for k, v in obesity_order.items()})

In [13]:
submission_df.set_index('id', inplace=True)

In [14]:
submission_df.to_csv('/kaggle/working/my_submission6.csv')

In [15]:
len(submission_df)

13840